In [1]:
#!pip install "numpy<2"

import torch
import torch.nn as nn
from torchvision import models, transforms
from PIL import Image

# Define model structure

In [2]:
# Model Definition
class KeyPointModel(nn.Module):
    def __init__(self):
        super(KeyPointModel, self).__init__()
        self.resnet = models.resnet50(pretrained=True)
        self.resnet.fc = nn.Linear(self.resnet.fc.in_features, 24)  # 12 key points (24 coordinates) - add fuly connected layer

    def forward(self, x):
        return self.resnet(x)


# load data

In [3]:
import os
import pandas as pd
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision import models
from PIL import Image
from torch.utils.data import Dataset, DataLoader

# Instantiate the model
model = KeyPointModel()
model.train()  # Set to training mode

# Dataset Definition
class KeyPointDataset(Dataset):
    def __init__(self, image_paths, key_points, transform=None):
        self.image_paths = image_paths
        self.key_points = key_points
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image = Image.open(self.image_paths[idx])
        key_point = self.key_points[idx]

        if self.transform:
            image = self.transform(image)

        return image, key_point

# Function to load dataset from a specified folder
def load_dataset(folder_path):
    # Load key points from the CSV file
    csv_file = os.path.join(folder_path, 'augmented_labels.csv')  # Change to your CSV file name
    keypoint_data = pd.read_csv(csv_file)

    # Create lists for images and key points
    image_paths = []
    key_points = []

    # Iterate through the CSV to get paths and key points
    for index, row in keypoint_data.iterrows():
        image_name = row['image_name']  # Replace with the actual column name in your CSV
        keypoint = row[1:].values.astype(float)  # Assuming the first column is the image name
        image_path = os.path.join(folder_path, f"{image_name}")  # Assuming images are in PNG format

        image_paths.append(image_path)
        key_points.append(torch.tensor(keypoint))

    return image_paths, key_points

# Base path for your project
base_path = os.path.expanduser('~/Documents/GitHub/ViT_facemap/ViT-pytorch/projects/Facemap/data')

# Paths to your train and test folders
train_folder = os.path.join(base_path, 'train', 'augmented_data')
test_folder = os.path.join(base_path, 'test', 'augmented_data')

# Load datasets
train_image_paths, train_key_points = load_dataset(train_folder)
test_image_paths, test_key_points = load_dataset(test_folder)

# Transformations
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Create DataLoaders
train_dataset = KeyPointDataset(train_image_paths, train_key_points, transform)
train_dataloader = DataLoader(train_dataset, batch_size=20, shuffle=True)

test_dataset = KeyPointDataset(test_image_paths, test_key_points, transform)
test_dataloader = DataLoader(test_dataset, batch_size=20, shuffle=False)

# Define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


C:\Users\avs20\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\avs20\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
# train network

In [4]:
import torch
import wandb

# Initialize WandB
wandb.init(project="Facemap_ResNet", name="10_epochs")

# Define your hyperparameters and log them to WandB
config = wandb.config
config.num_epochs = 50
config.learning_rate = 0.001
config.batch_size = 20

# Check if CUDA is available and set device accordingly
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Move model to the appropriate device
model = model.to(device)

# Training Loop
num_epochs = 50  # Number of epochs
config.num_epochs = num_epochs
for epoch in range(num_epochs):
    running_loss = 0.0
    for batch_idx, (images, targets) in enumerate(train_dataloader):
        # Move data to the GPU
        images = images.to(device)
        targets = targets.float().to(device)  
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()

        # Log the loss for the batch to WandB
        wandb.log({"Batch Loss": loss.item()})
        
        # Print progress every 10 batches (adjust as needed)
        if batch_idx % 10 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Batch [{batch_idx}/{len(train_dataloader)}], Loss: {loss.item():.4f}')

    # Print the average loss for the epoch
    avg_loss = running_loss / len(train_dataloader)
    print(f'Epoch [{epoch+1}/{num_epochs}], Average Loss: {avg_loss:.4f}')
    wandb.log({"Epoch": epoch + 1, "Average Loss": avg_loss})

# Save the trained model
model_path = 'keypoint_model.pth'
torch.save(model.state_dict(), model_path)

# Save the model to WandB
wandb.save(model_path)


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: anna-stuckert (anna-stuckert-university-of-st-andrews). Use `wandb login --relogin` to force relogin


Epoch [1/50], Batch [0/45], Loss: 10774.8916
Epoch [1/50], Batch [10/45], Loss: 8856.8779
Epoch [1/50], Batch [20/45], Loss: 5473.4971
Epoch [1/50], Batch [30/45], Loss: 3308.6038
Epoch [1/50], Batch [40/45], Loss: 1973.7061
Epoch [1/50], Average Loss: 5516.5233
Epoch [2/50], Batch [0/45], Loss: 1423.7258
Epoch [2/50], Batch [10/45], Loss: 852.3307
Epoch [2/50], Batch [20/45], Loss: 449.5571
Epoch [2/50], Batch [30/45], Loss: 282.5428
Epoch [2/50], Batch [40/45], Loss: 271.3300
Epoch [2/50], Average Loss: 588.4402
Epoch [3/50], Batch [0/45], Loss: 85.2262
Epoch [3/50], Batch [10/45], Loss: 56.2999
Epoch [3/50], Batch [20/45], Loss: 277.9798
Epoch [3/50], Batch [30/45], Loss: 288.2138
Epoch [3/50], Batch [40/45], Loss: 26.2006
Epoch [3/50], Average Loss: 102.4812
Epoch [4/50], Batch [0/45], Loss: 220.0223
Epoch [4/50], Batch [10/45], Loss: 28.1806
Epoch [4/50], Batch [20/45], Loss: 29.0820
Epoch [4/50], Batch [30/45], Loss: 38.3321
Epoch [4/50], Batch [40/45], Loss: 28.4790
Epoch [4/50]

['c:\\Users\\avs20\\Documents\\GitHub\\ViT_facemap\\ResNet50\\wandb\\run-20240927_130721-3ah26sbq\\files\\keypoint_model.pth']

# Plotting predictions via forward pass

In [ ]:
import torch
from torchvision import transforms
from PIL import Image
import matplotlib.pyplot as plt

# Load the trained model
model = KeyPointModel()
model.load_state_dict(torch.load('keypoint_model.pth'))
model.eval()  # Set to evaluation mode

# Function to preprocess an image
def preprocess_image(image_path):
    image = Image.open(image_path)
    transform = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    image = transform(image).unsqueeze(0)  # Add batch dimension
    return image

# Function to visualize predictions
def visualize_predictions(image_path, keypoints):
    # Load and display the original image
    original_image = Image.open(image_path)
    plt.imshow(original_image)
    
    # Convert keypoints to a numpy array and plot them
    x_coords = keypoints[0::2]
    y_coords = keypoints[1::2]
    plt.scatter(x_coords, y_coords, c='red', marker='x')
    plt.title('Predicted Key Points')
    plt.axis('off')
    plt.show()

# Path to the image you want to predict key points for
# Base path for your project
base_path = os.path.expanduser('~/Documents/GitHub/ViT_facemap/ViT-pytorch/projects/Facemap/data')

# Directory containing the images
image_folder = os.path.join(base_path, 'test', 'augmented_data')

# Get the first image in the folder
image_files = [f for f in os.listdir(image_folder) if f.endswith('.jpg') or f.endswith('.png')]
if not image_files:
    raise ValueError("No images found in the specified folder.")
image_path = os.path.join(image_folder, image_files[0])


# Preprocess the image
input_image = preprocess_image(image_path)

# Perform a forward pass to get predictions
with torch.no_grad():
    predicted_keypoints = model(input_image)

# Convert predictions to numpy array and detach from the graph
predicted_keypoints = predicted_keypoints.squeeze().numpy()

# Visualize the predicted key points
visualize_predictions(image_path, predicted_keypoints)


# SHAP maybe works?

In [ ]:
#!pip install shap
import os
import shap
import numpy as np
import torch
from torchvision import transforms
from PIL import Image

# Load the trained model
model = KeyPointModel()
model.load_state_dict(torch.load('keypoint_model.pth'))
model.eval()

# Prepare the transform
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Function to prepare input for SHAP
def preprocess_image(image_path):
    image = Image.open(image_path)
    image = transform(image).unsqueeze(0)  # Add batch dimension
    return image

# Directory containing the images
# Base path for your project
base_path = os.path.expanduser('~/Documents/GitHub/ViT_facemap/ViT-pytorch/projects/Facemap/data')

# Directory containing the images
image_folder = os.path.join(base_path, 'test', 'augmented_data')

# Get the first image in the folder
image_files = [f for f in os.listdir(image_folder) if f.endswith('.jpg') or f.endswith('.png')]
if not image_files:
    raise ValueError("No images found in the specified folder.")
first_image_path = os.path.join(image_folder, image_files[0])

# Prepare the image for SHAP analysis
images = preprocess_image(first_image_path)

# Reshape images to 2D (num_samples, num_features)
images_reshaped = images.view(images.size(0), -1)  # Flatten each image

# Create a SHAP explainer
def model_predict(input_data):
    input_data = torch.tensor(input_data, dtype=torch.float32)  # Ensure input is a tensor
    with torch.no_grad():
        return model(input_data.view(-1, 3, 224, 224)).numpy()  # Reshape back to original image size

# Initialize the Kernel Explainer with flattened images
explainer = shap.KernelExplainer(model_predict, images_reshaped.numpy())

# Calculate SHAP values
shap_values = explainer.shap_values(images_reshaped.numpy())

# Visualize SHAP values for the first image
shap.initjs()

# Remove the batch dimension for visualization
images_for_plotting = images.squeeze(0).permute(1, 2, 0).numpy()  # Shape to (224, 224, 3)

shap.image_plot(shap_values, images_for_plotting)


# SHAP based on each KP

https://shap.readthedocs.io/en/latest/example_notebooks/image_examples/image_classification/Explain%20ResNet50%20using%20the%20Partition%20explainer.html 

In [6]:
import os
import pandas as pd
import torch
from torchvision import transforms
from PIL import Image

# Load the trained model
model = KeyPointModel()
model.load_state_dict(torch.load('keypoint_model.pth'))
model.eval()

# Function to load dataset from a specified folder
def load_dataset(folder_path):
    # Load key points from the CSV file
    csv_file = os.path.join(folder_path, 'augmented_labels.csv')  # Change to your CSV file name
    keypoint_data = pd.read_csv(csv_file)

    # Create lists for images and key points
    image_paths = []
    key_points = []

    # Iterate through the CSV to get paths and key points
    for index, row in keypoint_data.iterrows():
        image_name = row['image_name']  # Replace with the actual column name in your CSV
        keypoint = row[1:].values.astype(float)  # Assuming the first column is the image name
        image_path = os.path.join(folder_path, f"{image_name}")  # Assuming images are in PNG format

        image_paths.append(image_path)
        key_points.append(torch.tensor(keypoint))

    return image_paths, key_points

# Function to load and preprocess images
def load_and_preprocess_images(image_paths):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),  # Resize to model's input size
        transforms.ToTensor(),  # Convert to tensor
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize for ResNet
    ])
    
    images = []
    for path in image_paths:
        image = Image.open(path).convert('RGB')  # Load image and convert to RGB
        image = transform(image)  # Apply transformations
        images.append(image)
    
    return torch.stack(images)  # Stack images into a single tensor

# Base path for your project
base_path = os.path.expanduser('~/Documents/GitHub/ViT_facemap/ViT-pytorch/projects/Facemap/data')

# Paths to your train and test folders
train_folder = os.path.join(base_path, 'train', 'augmented_data')
test_folder = os.path.join(base_path, 'test', 'augmented_data')

# Load datasets
train_image_paths, train_key_points = load_dataset(train_folder)
test_image_paths, test_key_points = load_dataset(test_folder)

# Load and preprocess images
X_train = load_and_preprocess_images(train_image_paths)
X_test = load_and_preprocess_images(test_image_paths)

# Convert key points to tensors
y_train = torch.stack(train_key_points)
y_test = torch.stack(test_key_points)


In [ ]:
import shap
import numpy as np
import torch

# Check if CUDA is available and set device accordingly
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Move model to the appropriate device
model = model.to(device)

# Assuming 'X_train' is your training data
# Ensure that X_train is a PyTorch tensor and move it to the GPU
X_train_tensor = torch.tensor(X_train).to(device)

# Use a very small sample of data to avoid OOM error
X_train_sample = X_train_tensor[:10]  # Adjust based on your needs

# Clear GPU memory
torch.cuda.empty_cache()

# Create the SHAP explainer
explainer = shap.GradientExplainer(model.cpu(), X_train_sample.cpu())  # Use CPU for SHAP

# Use torch.no_grad() to save memory during SHAP value computation
with torch.no_grad():
    shap_values = explainer.shap_values(X_train_sample.cpu())

# Convert SHAP values back to numpy if needed
shap_values_np = [val.detach().numpy() for val in shap_values]

# Now you can use shap_values_np for further analysis or visualization





In [ ]:
# 'X_test' is your test data for which you want explanations
shap_values = explainer.shap_values(X_test)


In [ ]:
# Compute SHAP values in smaller batches
shap_values = []
batch_size = 1  # Adjust this value based on your available memory

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
X_test = X_test.to(device)


with torch.no_grad():
    for i in range(0, X_test.shape[0], batch_size):
        batch_shap_values = explainer.shap_values(X_test[i:i + batch_size])
        shap_values.extend(batch_shap_values)
        torch.cuda.empty_cache()  # Clear memory after each batch if necessary



In [ ]:
for i in range(12):  # Loop through each key point
    shap.initjs()
    shap.summary_plot(shap_values[i], X_test, feature_names=your_feature_names)
